In [12]:
from sentence_transformers import SentenceTransformer
import numpy as np
import requests 

In [2]:
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

## Q1: What's the first value of the resulting vector?

In [5]:
user_question = "I just discovered the course. Can I still join it?"
embdedding=embedding_model.encode(user_question)
embdedding[0]

0.078222655

### Preparing the documents

In [16]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [17]:
course='machine-learning-zoomcamp'
course_documents=[doc for doc in documents if doc['course']==course]


25


## Q2: What's the shape of X?